In [1]:
from os import listdir
from os.path import splitext
from os.path import join

import skvideo
skvideo.setFFmpegPath(r'C:\Users\ion\anaconda3\envs\tf-gpu\Lib\site-packages\ffmpeg-4.3.1-2020-10-01-full_build\bin')
import skvideo.io
import json
import cv2

## Explanation of different datasets
The Aff-Wild2 dataset is used for training, validation and testing. While the AFEW 7.0 dataset is only used for testing. 

#### Aff-Wild2
In this dataset all videos are in one folder. Each video has a corresponding .txt file where each frame is annotated with the correspending facial expression class at that frame. In each video multiple emotions are displayed, thats why the annotation is per frame.

In [2]:
# Aff-Wild2 = AW2
# filenames = FN

### Aff-wild2 ###
# Directories for labels/annotations for each set
AW2_dir_train_labels = r"D:\Aff-Wild2 Dataset\Aff-wild2\Videos\annotations\EXPR_Set\Training_Set"
AW2_dir_val_labels = r"D:\Aff-Wild2 Dataset\Aff-wild2\Videos\annotations\EXPR_Set\Validation_Set"

# Directories for videos
AW2_dir_allvideos = r"D:\Aff-Wild2 Dataset\Aff-wild2\Videos\all_videos"

# Save the filenames in a list for each set, and split the filename from the extension
AW2_train_FN = listdir(AW2_dir_train_labels)
AW2_train_FN_split = [splitext(file) for file in AW2_train_FN]

AW2_val_FN = listdir(AW2_dir_val_labels)
AW2_val_FN_split = [splitext(file) for file in AW2_val_FN]

AW2_videos_FN = listdir(AW2_dir_allvideos)
AW2_videos_FN_split = [splitext(file) for file in AW2_videos_FN]

### Aff-wild2 Dataset Size ###

In [3]:
# Size of each set
print(f"The training-set contains {len(AW2_train_FN)} videos")
print(f"The validation-set contains {len(AW2_val_FN)} videos")
print(f"The test-set contains {len(AW2_videos_FN) - (len(AW2_train_FN) + len(AW2_val_FN))} videos")
print(f"The whole dataset contains {len(AW2_videos_FN)} videos")

The training-set contains 253 videos
The validation-set contains 70 videos
The test-set contains 225 videos
The whole dataset contains 548 videos


In [4]:
# Checking if the labeled filenames corresponds to the videos
train_size = 0
for i in AW2_train_FN_split:
    for j in AW2_videos_FN_split:
        if i[0] == j[0]:
            train_size+=1
    
## The results show that only 246 labels are matched to videos, while there were 253 labeled videos
## Some videos contain two persons, so the labels are for one person on the left or right
print(train_size)

246


In [5]:
# Checking if the labeled filenames corresponds to the videos (and disregarding if its on left or right)
train_size = 0
for i in AW2_train_FN_split:
    for j in AW2_videos_FN_split:
        if '_left' in i[0]:
            a = i[0].replace('_left', '')
        elif '_right' in i[0]:
            a = i[0].replace('_right', '')
        else:
            a = i[0]
        if a == j[0]:
            train_size+=1
# Labels is equal to videos now
print(f'Training dataset is indeed = {train_size} videos')

# Checking if the labeled filenames corresponds to the videos (and disregarding if its on left or right)
val_size = 0
for i in AW2_val_FN_split:
    for j in AW2_videos_FN_split:
        if '_left' in i[0]:
            a = i[0].replace('_left', '')
        elif '_right' in i[0]:
            a = i[0].replace('_right', '')
        else:
            a = i[0]
        if a == j[0]:
            val_size+=1
# Labels is equal to videos now
print(f'Validation dataset is indeed = {val_size} videos')

Training dataset is indeed = 253 videos
Validation dataset is indeed = 70 videos


### Sequence length of each video ###

In [6]:
# Create dictionary of sequence length of each video with ffprobe in skvideo
AW2_seqlength = {}

for video in AW2_videos_FN:
    metadata = skvideo.io.ffprobe(join(AW2_dir_allvideos, video))
    frames = int(metadata.get('video').get('@nb_frames'))
    width = int(metadata.get('video').get('@width'))
    height = int(metadata.get('video').get('@height'))
    AW2_seqlength[video] = [frames, height, width, 3]

In [60]:
# Create dictionary of sequence length of each video with ffprobe in skvideo and opencv to capture the amount of frames
AW2_seqlength_V2 = {}

for video in AW2_videos_FN:
    metadata = skvideo.io.ffprobe(join(AW2_dir_allvideos, video))
    width = int(metadata.get('video').get('@width'))
    height = int(metadata.get('video').get('@height'))
    
    cap = cv2.VideoCapture(join(AW2_dir_allvideos, video))
    frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    AW2_seqlength_V2[video] = [frames, height, width, 3]

In [ ]:
# Create dictionary of sequence length of each video with ffprobe in skvideo and opencv to capture the amount of frames manually
AW2_seqlength_V3 = {}

counter = 0
for video in AW2_videos_FN:
    print(counter)
    counter+=1
    metadata = skvideo.io.ffprobe(join(AW2_dir_allvideos, video))
    width = int(metadata.get('video').get('@width'))
    height = int(metadata.get('video').get('@height'))
    
    frames = count_frames_manual(cv2.VideoCapture(join(AW2_dir_allvideos,video)))
    AW2_seqlength_V3[video] = [frames, height, width, 3]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124


In [61]:
# Checking if the video length is the same with each method
AW2_seqlength == AW2_seqlength

True

In [ ]:
# Save json file with all video's names and shapes = (n_frames, width, height, colour channels)
with open('AW2_video_shapes.json', 'w') as fp:
    json.dump(AW2_seqlength, fp)

In [62]:
# Load Aff-Wild2 video_shapes.json
with open('AW2_video_shapes.json', 'r') as fp:
    AW2_video_shapes = json.load(fp)